In [ ]:
import csv
import time
from bs4 import BeautifulSoup
import os
import requests
from csv import writer
import random
import json

In [2]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36',
    'cookie': 'bid=3sjUyZu9BhY; douban-fav-remind=1; viewed="1007305"; _pk_id.100001.4cf6=fe0c750fe7d5ebcc.1732548765.; __yadk_uid=3y2FwpVwXLd6GATDtwe9eh3G63rXgVvW; ll="118096"; _vwo_uuid_v2=D4DE1E32F6CDDB3CABACBC07C7B7AAD0A|4d1e71856bd86a1584f8a9ea79f4b2ff; push_noty_num=0; push_doumail_num=0; _cc_id=bf9e7c5bfcf0ed75d27c4d16d3ce1601; _ga=GA1.2.1068251831.1736592634; _ga_Y4GN1R87RG=GS1.1.1736596766.2.0.1736596766.0.0.0; __utmv=30149280.18192; __utmz=30149280.1737736843.11.8.utmcsr=cn.bing.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmz=223695111.1737736843.9.6.utmcsr=cn.bing.com|utmccn=(referral)|utmcmd=referral|utmcct=/; dbcl2="181926153:/OhktQsTDnw"; panoramaId_expiry=1739340770924; panoramaId=1538ceb9a89ae22e4b5caad643cfa9fb927a5e68f9f8a016d8642467ea14a965; panoramaIdType=panoDevice; __gads=ID=35042766bd7ede1e:T=1736346898:RT=1739255053:S=ALNI_MYaCIiZpgN-8ULWSWtEbHhDXuCdEg; __gpi=UID=00000fb1bdf1c178:T=1736346898:RT=1739255053:S=ALNI_Mag95R6tJEOb80jn_SWEPDjr6CHcg; __eoi=ID=70358b6f5ef27592:T=1736346898:RT=1739255053:S=AA-AfjbtOT9g2EmShnzX2bpo3AEi; FCNEC=%5B%5B%22AKsRol_usVaqL2YHzgMekBu_syJ6DUOMicT6kVaUF6LjhZTHfA-AxwIQG1YYjJaQMpcEBkJ2sKCVfJOzji8EIlkh4ByRWfd7UBRa-tM9FJtFKCSxdLoMI34ndyugbfGSSZOE5Z2qsPEiZgR74SA-8I_VMaO85MZPPw%3D%3D%22%5D%5D; ck=d2Qp; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1739266827%2C%22https%3A%2F%2Fcn.bing.com%2F%22%5D; _pk_ses.100001.4cf6=1; __utma=30149280.799383041.1726306427.1739252183.1739266828.14; __utmb=30149280.0.10.1739266828; __utmc=30149280; __utma=223695111.1512467655.1732548765.1739252186.1739266828.12; __utmb=223695111.0.10.1739266828; __utmc=223695111; frodotk_db="d77f5cf912308a00867e16eb36919e00"'
}

In [3]:
# 创建保存目录
directory = "电影短评"
if not os.path.exists(directory):
    os.makedirs(directory)
# 创建保存目录
directorys = "电影详情"
if not os.path.exists(directorys):
    os.makedirs(directorys)

In [4]:
def get_movie_ids(num_ids=150):
    """获取电影ID列表"""
    movie_ids = set()
    
    page = 0
    while len(movie_ids) < num_ids:
        try:
            urls = [
                f'https://movie.douban.com/j/search_subjects?type=movie&tag=热门&sort=recommend&page_limit=20&page_start={page}',
                f'https://movie.douban.com/j/search_subjects?type=movie&tag=最新&sort=time&page_limit=20&page_start={page}',
                f'https://movie.douban.com/j/search_subjects?type=movie&tag=经典&sort=rank&page_limit=20&page_start={page}'
            ]
            
            for url in urls:
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    data = response.json()
                    for movie in data.get('subjects', []):
                        movie_ids.add(movie['id'])
                        
                        if len(movie_ids) >= num_ids:
                            break
                
                time.sleep(random.uniform(1, 2))
            
            page += 20
            
        except Exception as e:
            print(f"获取电影ID时出错: {e}")
            time.sleep(3)
            continue
            
        if page > 200:
            break
    
    return list(movie_ids)

In [5]:
def get_star_rating(field):
    stars = {
        'allstar10': '1星',
        'allstar20': '2星',
        'allstar30': '3星',
        'allstar40': '4星',
        'allstar50': '5星'
    }
    return stars.get(field, '未评分')

In [6]:
def get_name(movie_ID):
    url = f'https://movie.douban.com/subject/{movie_ID}/comments'
    res = requests.get(url, headers=headers)
    bs = BeautifulSoup(res.text, 'html.parser')
    return bs.find('h1').get_text()

In [7]:
def save_data(comment, writer):
    info = comment.find('span', {'class': 'comment-info'})
    data_dict = {
        "用户名": info.find('a').get_text().strip(),
        "评论时间": info.find('span', {'class': 'comment-time'}).get_text().strip(),
        "IP属地": info.find('span', {'class': 'comment-location'}).get_text().strip(),
        "评论分数": get_star_rating(info.find_all('span')[1]['class'][0]),
        "有用数量": comment.find('span', {'class': 'votes vote-count'}).get_text().strip(),
        "短评内容": comment.find('span', {'class': 'short'}).get_text().replace('\n', '').strip().replace('\r', '').replace('\t', ''),
    }
    
    global short_count
    short_count += 1

    print(f"当前评论数量：{short_count}\n",
          f"用户名：{data_dict['用户名']}\n",
          f"评论时间：{data_dict['评论时间']}\n",
          f"IP属地：{data_dict['IP属地']}\n",
          f"评论分数：{data_dict['评论分数']}\n",
          f"有用数量：{data_dict['有用数量']}\n",
          f"短评内容：{data_dict['短评内容']}\n")
    
    try:
        writer.writerow(data_dict)
        return True
    except Exception as e:
        print(f"保存评论时出错: {e}")
        return False

In [8]:
def get_comments(movie_ID):
    """获取电影评论"""
    page_count = 0
    next_url = ''
    
    while True and short_count < 100:
        time.sleep(2)
        url = f'https://movie.douban.com/subject/{movie_ID}/comments{next_url}'
        try:
            res = requests.get(url, headers=headers)
            bs = BeautifulSoup(res.text, 'html.parser')
            page = bs.find('div', {'id': 'comments'})
            
            if page is None:
                if short_count == 0:
                    return False
                break
                
            comments = page.find_all('div', {'class': "comment-item"})
            for comment in comments:
                if short_count >= 100:
                    return True
                save_data(comment, comment_writer)
            
            print('====================爬取page{}完毕==================='.format(page_count))
            page_count += 1
            
            try:
                next_url = page.find('div', {'id': 'paginator', 'class': 'center'}).find('a', {'data-page': 'next', 'class': 'next'}).attrs["href"]
            except:
                print('爬取完毕')
                break
                
        except Exception as e:
            print(f"获取评论时出错: {e}")
            if short_count == 0:
                return False
            break
    
    return True


In [9]:
def check_existing_movies():
    """检查已存在的电影CSV文件"""
    successful_count = 0
    existing_names = set()
    
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            try:
                with open(os.path.join(directory, filename), 'r', encoding='utf-8-sig') as f:
                    reader = csv.reader(f)
                    next(reader)  # 跳过标题行
                    comment_count = sum(1 for _ in reader)
                    if comment_count >= 100:
                        successful_count += 1
                        existing_names.add(filename[:-4])  # 去掉.csv后缀
                        print(f"已验证电影 {filename[:-4]} 的数据完整性")
            except Exception as e:
                print(f"验证电影 {filename} 时出错: {e}")
    
    return successful_count, existing_names

In [10]:
def save_to_json(movies, filename='movies.json'):
    """保存电影详情到JSON文件"""
    filepath = os.path.join(directorys, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(movies, f, ensure_ascii=False, indent=2)

In [11]:
def get_movie_details(movie_id):
    """获取电影详情"""
    url = f'https://movie.douban.com/subject/{movie_id}/'
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 获取标题
        title = soup.find('span', property='v:itemreviewed').text.strip()
        
        # 获取评分
        rating = soup.find('strong', property='v:average').text.strip()
        
        # 获取制片国家/地区
        info = soup.find('div', {'id': 'info'}).text
        country_start = info.find('制片国家/地区:') + len('制片国家/地区:')
        country_end = info.find('\n', country_start)
        countries = info[country_start:country_end].strip()
        
        # 获取类型
        genres = []
        genre_spans = soup.find_all('span', property='v:genre')
        for span in genre_spans:
            genres.append(span.text.strip())
        
        return {
            'title': title,
            'rating': rating,
            'countries': countries,
            'genres': genres 
        }
    except Exception as e:
        print(f"获取电影 {movie_id} 详情时出错: {e}")
        return None


In [12]:
def main():
    # 获取电影ID
    movie_ids = get_movie_ids(150)
    movies = []
    
    print(f"开始获取 {len(movie_ids)} 部电影的详情")
    
    for i, movie_id in enumerate(movie_ids, 1):
        print(f"\n正在处理第 {i} 部电影 (ID: {movie_id})")
        
        details = get_movie_details(movie_id)
        if details:
            movies.append(details)
            print(f"成功获取电影信息: {details['title']}")
            print(f"类型: {', '.join(details['genres'])}")  # 打印类型信息
        
        # 随机延时
        time.sleep(random.uniform(2, 4))
    
    # 保存结果
    save_to_json(movies)
    print(f"\n完成! 共获取 {len(movies)} 部电影的详情")

In [ ]:
if __name__ == '__main__':
    # 检查已有的电影数据
    successful_movies, existing_names = check_existing_movies()
    print(f"已找到 {successful_movies} 部完整电影数据")
    
    if successful_movies >= 100:
        print("已经完成100部电影的数据收集")
        exit(0)
    
    # 获取新的电影ID
    remaining_movies = 100 - successful_movies
    all_movie_ids = get_movie_ids(remaining_movies + 50)  # 多获取一些作为备用
    current_index = 0
    
    while successful_movies < 100 and current_index < len(all_movie_ids):
        movie_ID = all_movie_ids[current_index]
        current_index += 1
        
        try:
            # 获取电影名称
            name = get_name(movie_ID)
            print(f"\n开始处理电影: {name}")
            
            # 检查是否已经存在相同名称的电影
            if name in existing_names:
                print(f"跳过重复电影: {name}")
                continue
            
            # 获取评论
            short_count = 0
            comment_file = os.path.join(directory, f"{name}.csv")
            
            with open(comment_file, 'w', encoding="utf-8-sig", newline="") as f:
                header = ["用户名", "评论时间", "IP属地", "评论分数", "有用数量", "短评内容"]
                comment_writer = csv.DictWriter(f, header)
                comment_writer.writeheader()
                print(f'==============正在爬取{name}短评====================')
                
                if get_comments(movie_ID):
                    successful_movies += 1
                    existing_names.add(name)
                    print(f"成功获取第 {successful_movies} 部电影的数据")
                else:
                    print(f"跳过电影 {name}，无法获取评论")
                    # 删除空的CSV文件
                    if os.path.exists(comment_file):
                        os.remove(comment_file)
        
        except Exception as e:
            print(f"处理电影ID {movie_ID} 时出错: {e}")
            continue
        
        time.sleep(random.uniform(2, 4))
    
    print(f"\n最终成功获取 {successful_movies} 部电影的数据")
    main()